In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
import pandas as pd 
from selenium.webdriver.chrome.options import Options
import os 
from glob import glob
from io import StringIO 
import shutil
import uuid

download_folder = r"D:\gtrends-scraping\scraped_data_temp"  # Change this to your desired download path

chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,  # To prevent the download prompt
    "download.directory_upgrade": True,
})

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

In [2]:
def make_temp_folder(download_folder = download_folder):
    if not os.path.exists(download_folder): 
        os.makedirs(download_folder)
make_temp_folder()

In [3]:
driver.get('https://trends.google.com/trends/explore?geo=IN&hl=en')
driver.maximize_window()
driver.refresh()

In [4]:
try:
    cookie_button = driver.find_element(By.XPATH,'//*[@id="cookieBar"]/div/span[2]/a[2]') 
    cookie_button.click()
except Exception as e: 
    print(e)

In [5]:
KEYS_TO_SEND = 'pollution'
COUNTRY_NAME = 'Indonesia'

In [6]:
search_term = driver.find_element(By.XPATH,'//*[@id="input-29"]')
search_term.send_keys(KEYS_TO_SEND)
search_term.send_keys(Keys.RETURN)

In [7]:
def search_country_name(country_name): 
    country_search = driver.find_element(By.XPATH,'//*[@id="compare-pickers-wrapper"]/div/hierarchy-picker[1]/ng-include/div[1]')
    country_search.click()
    send_country_name = driver.find_element(By.XPATH,'//*[@id="input-10"]')
    send_country_name.send_keys(country_name)
    send_country_name.send_keys(Keys.ARROW_DOWN)
    send_country_name.send_keys(Keys.RETURN)



In [8]:
search_country_name(COUNTRY_NAME)

In [9]:
month_picker = driver.find_element(By.XPATH,'//*[@id="compare-pickers-wrapper"]/div/custom-date-picker')
month_picker.click()

# month_dropdown = driver.find_element(By.XPATH,'//*[@id="select_option_24"]')
# month_dropdown.click()

In [10]:
date_five_year_element = driver.find_element(
    By.XPATH, '//div[contains(text(),"Past 5 years")]'
)
date_five_year_element.click()

In [14]:
import shutil

def remove_temp_folder(download_folder):
    try:
        shutil.rmtree(download_folder)
    except FileNotFoundError: 
        pass 



In [15]:
related_topics_download = driver.find_element(
    By.XPATH,
    "/html/body/div[2]/div[2]/div/md-content/div/div/div[3]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]",
)

related_topics_download.click()

related_queries_download = driver.find_element(
    By.XPATH, 
    "/html/body/div[2]/div[2]/div/md-content/div/div/div[4]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]"
)

related_queries_download.click()

In [21]:
import pandas as pd

def process_lines(lines):
    top_index = lines.index("TOP\n")
    rising_index = lines.index("RISING\n")

    top_values = lines[top_index : rising_index - 1]
    rising_values = lines[rising_index : len(lines)]

    top_values_text = "".join(top_values)
    rising_values_text = "".join(rising_values)

    rising_df = pd.read_csv(
        StringIO(rising_values_text),
    ).reset_index()
    rising_df.columns = ["TERM", "CHANGE"]
    rising_df["IDENTIFIER"] = "RISING"

    top_df = pd.read_csv(
        StringIO(top_values_text),
    ).reset_index()
    top_df.columns = ["TERM", "CHANGE"]
    top_df["IDENTIFIER"] = "TOP"

    return top_df, rising_df

In [23]:
def process_related_topics():
    with open(download_folder + r"\relatedEntities.csv") as file:

        lines = file.readlines()
        top_df, rising_df = process_lines(lines)

    df = pd.concat([top_df, rising_df])
    df['RELATED_TYPE'] = 'RELATED_TOPICS'
    return df 


def process_related_queries():
    with open(download_folder + r"\relatedQueries.csv") as file:
        lines = file.readlines()
        top_df, rising_df = process_lines(lines)

    df = pd.concat([top_df, rising_df])
    df['RELATED_TYPE'] = 'RELATED_QUERIES'
    return df 

In [24]:
df = pd.concat([process_related_queries(), process_related_topics()])

In [27]:
trend_data = driver.find_element(By.XPATH,"/html/body/div[2]/div[2]/div/md-content/div/div/div[1]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]")
trend_data.click()

In [30]:
trends_data = rf"{download_folder}\multiTimeline.csv"

In [31]:
trends_df = pd.read_csv(trends_data, skiprows=2)

In [32]:
trends_df

,Week,pollution: (Indonesia)
0,2019-03-17,57
1,2019-03-24,53
2,2019-03-31,36
3,2019-04-07,47
4,2019-04-14,39
...,...,...
257,2024-02-18,45
258,2024-02-25,51
259,2024-03-03,47
260,2024-03-10,30
